## Предсказание жилых зданий


In [1]:
from osm_living_predictor.downloader import OSMDownloader
from osm_living_predictor.building_processor import BuildingProcessor
from osm_living_predictor.road_processor import RoadProcessor
from osm_living_predictor.amenity_processor import AmenityProcessor
from osm_living_predictor.feature_builder import FeatureBuilder
from osm_living_predictor.model_handler import ModelHandler


In [2]:
downloader = OSMDownloader(12030887)
bounds = downloader.load_boundary()

In [3]:
# Обработка
buildings = BuildingProcessor(bounds)
buildings.load_buildings()

In [4]:
roads = RoadProcessor(bounds, buildings.buildings)
roads.load_roads()

In [5]:
amenities = AmenityProcessor(bounds, roads.backup_data)
amenities.load_amenities()

In [6]:
# Признаки
builder = FeatureBuilder(buildings, roads, amenities)
data = builder.build_features()

d:\programming\github\floor-predictor\osm_living_predictor\feature_builder.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['landuse'].replace([


In [ ]:
# Создание модели

from sklearn.ensemble import RandomForestClassifier

external_model = RandomForestClassifier(random_state=42)
handler = ModelHandler("model/model_dt.pkl", df=data, target_col="is_living")
handler.set_model(external_model)

X_train, X_test, y_train, y_test = handler.train_test_split()
handler.train_model(X_train, y_train)

predicted = handler.predict(X_test, map_labels=True)




[ModelHandler] Внешняя модель установлена.
[CV] Accuracy: mean=0.8293, std=0.0031
[ModelHandler] Модель сохранена: model_dt.pkl


In [7]:
# Загрузка модели

handler = ModelHandler("model/model_dt.pkl", df=data, target_col="is_living")
handler.load_model_from_file()

predicted = handler.predict(data, map_labels=False)

[ModelHandler] Модель успешно загружена из файла.


## Предсказание этажности


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from osm_height_predictor.geo import (BuildingPreprocessor, 
                                GeometryFeatureGenerator, 
                                SpatialStatisticsComputer, 
                                SpatialNeighborhoodAnalyzer, 
                                StoreyModelTrainer)

In [ ]:
# Define the folder path
folder_path = "data/data_spb+towns/input_data_for_train"

pkl_files = [
    os.path.join(folder_path, f)
    for f in os.listdir(folder_path)
    if f.endswith(".pickle")
]

df_list = [pd.read_pickle(f).to_crs(4326) for f in pkl_files]
df_building = (
    pd.concat(df_list, ignore_index=True)
    .rename(columns={"building:levels": "storey"})
    .dropna(subset=["storey"])
)

df_building["storey"] = df_building["storey"].astype(int)

df_building = df_building[df_building["is_living"] == 1].reset_index(drop=True)
df_building = df_building[(df_building["storey"] > 2)]


print(f"Loaded {len(df_list)} files.")
print(df_building.info())

In [ ]:
# 1. Предобработка
prep = BuildingPreprocessor(df_building)
prep.filter_residential()
df = prep.get()

In [ ]:
# 2. Геометрические признаки
geo_gen = GeometryFeatureGenerator(df)
df = geo_gen.compute_geometry_features()

In [ ]:
# 3. Пространственный анализ
stats = SpatialStatisticsComputer(df)
df, global_moran, lisa = stats.compute_moran_and_lisa(col="storey")

In [ ]:
# 4. Соседние признаки
analyzer = SpatialNeighborhoodAnalyzer(df)
df = analyzer.compute_neighborhood_metrics()

In [ ]:
# 5. Обучение модели
trainer = StoreyModelTrainer(df)
X_train, X_test, y_train, y_test = trainer.prepare_data()
param_dist = {
                "n_estimators": [100],
            }

model = trainer.train_rf(X_train, y_train, param_dist=param_dist)
y_pred = model.predict(X_test)

## Plot

In [ ]:
b_gem = df.to_crs(3857).centroid.copy()

def plot_target_vs_prediction(
    y_true, y_pred, bins=30, title="RF Target vs Prediction Histogram"
):
    plt.figure(figsize=(10, 6))
    plt.hist(y_true, bins=bins, alpha=0.6, label="True (Target)", density=True)
    plt.hist(y_pred, bins=bins, alpha=0.6, label="Predicted", density=True)
    plt.xlabel("Value")
    plt.ylabel("Density")
    plt.title(title)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()


plot_target_vs_prediction(y_test, y_pred)

In [ ]:
import matplotlib.pyplot as plt

# Count occurrences of each (true, predicted) pair
counts = df_building.groupby(["storey", "pred_storey"]).size().reset_index(name="count")
import matplotlib.pyplot as plt
import numpy as np

# Normalize counts for bubble size (square root scaling helps perception)
counts["size"] = counts["count"] / 2  # tweak factor 30 as needed

plt.figure(figsize=(10, 7))
sc = plt.scatter(
    counts["storey"],
    counts["pred_storey"],
    s=counts["size"],
    alpha=0.3,
    c=np.log(counts["count"]),
    cmap="Blues",
    edgecolor="black",
    linewidth=0.8,
)

plt.plot(
    [df_building["storey"].min(), df_building["storey"].max()],
    [df_building["storey"].min(), df_building["storey"].max()],
    linestyle="--",
    color="red",
    label="Ideal (y = x)",
)

plt.xlabel("True number of storeys")
plt.ylabel("Predicted number of storeys")
plt.title("Predicted vs. True Storeys")
cbar = plt.colorbar(sc)
cbar.set_label("Log frequency")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from libpysal.weights import KNN
import geopandas as gpd

def analyze_residuals(gdf, true_col="storey", pred_col="pred_storey", k=10):
    """
    Analyzes spatial autocorrelation in model residuals
    """
    from esda.moran import Moran
    from libpysal.weights import KNN
    import matplotlib.pyplot as plt
    from splot.esda import plot_moran

    # Calculate residuals
    gdf["residuals"] = gdf[true_col] - gdf[pred_col]

    # Create spatial weights
    w = KNN.from_dataframe(gdf, k=k)
    w.transform = "r"

    # Calculate Moran's I for residuals
    moran = Moran(gdf["residuals"], w)

    print(f"\nResidual Analysis:")
    print(f"Moran's I: {moran.I:.4f}")
    print(f"p-value: {moran.p_sim:.4f}")

    # Plot Moran scatter plot
    fig, ax = plt.subplots(figsize=(12, 3))
    plot_moran(moran, zstandard=True)
    plt.title("Moran's I Scatter Plot for Model Residuals")
    plt.tight_layout()
    plt.show()

    return moran


# After model training, analyze residuals
moran_residuals = analyze_residuals(
    gpd.GeoDataFrame(df_building.join(b_gem.rename("geometry"), how="inner")),
    true_col="storey",
    pred_col="pred_storey",
    k=10,
)


In [ ]:
def plot_error_map(df, true_col="storey", pred_col="pred_storey"):
    """
    Plots a map showing prediction errors for building heights.
    Colors indicate if predictions are within ±20% of true values.
    'Within ±20%' predictions are shown with a glass-like transparency effect.
    """
    # Convert to GeoDataFrame if not already
    gdf = gpd.GeoDataFrame(df) if not isinstance(df, gpd.GeoDataFrame) else df

    # Calculate relative errors
    gdf["rel_error"] = round((gdf[pred_col] - gdf[true_col]) / gdf[true_col], 2)
    gdf["error"] = round(gdf[pred_col] - gdf[true_col], 2)

    # Create error categories
    conditions = [
        (gdf["rel_error"] < -0.15) & (np.abs(gdf["error"]) > 2),
        (gdf["rel_error"] > 0.15) & (np.abs(gdf["error"]) > 2),
        (gdf["rel_error"].between(-0.15, 0.15)),
    ]
    choices = ["Underprediction >15%", "Overprediction >15%", "Within ±15%"]
    gdf["error_category"] = np.select(conditions, choices, default="Within ±15%")

    # Create color mapping with transparency
    color_dict = {
        "Underprediction >15%": "red",
        "Overprediction >15%": "blue",
        "Within ±15%": "#7fcdbb",  # Light blue-green for glass effect
    }

    alpha_dict = {
        "Underprediction >15%": 0.7,
        "Overprediction >15%": 0.7,
        "Within ±15%": 0.3,  # More transparent for glass effect
    }

    # Create the plot
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))

    # Plot background layer first (all buildings with very light color)
    gdf.plot(color="lightgray", alpha=0.1, ax=ax)

    # Plot with categorical colors - plot good predictions first
    categories = ["Within ±15%", "Underprediction >15%", "Overprediction >15%"]
    for category in categories:
        mask = gdf["error_category"] == category
        if mask.any():
            gdf[mask].plot(
                color=color_dict[category],
                ax=ax,
                label=category,
                alpha=alpha_dict[category],
                edgecolor="white" if category == "Within ±15%" else "none",
                linewidth=0.5 if category == "Within ±15%" else 0,
                markersize=2,
            )

    # Customize the plot
    ax.set_title("Building Height Prediction Errors", pad=20)
    ax.axis("off")

    # Customize legend
    ax.legend(
        title="Prediction Accuracy", frameon=True, framealpha=0.9, edgecolor="white"
    )

    plt.tight_layout()
    plt.show()

    return gdf


# Use the function
gdf = plot_error_map(df_building.join(b_gem.rename("geometry"), how="inner"))


In [ ]:
gdf["abs_error"] = np.abs(gdf["storey"] - gdf["pred_storey"])
gdf["abs_error"] = gdf["abs_error"].round(3)

gdf["abs_rel_error"] = gdf["abs_error"] / (gdf["storey"] + 1e-6)
gdf["abs_rel_error"] = gdf["abs_rel_error"].round(3)

gdf.loc[
    (gdf["abs_error"] < 5) & (gdf["storey"] > 15),
    list(gdf.columns[-6:])
    + [
        "storey",
        "pred_storey",
        "cluster_High-High",
        "cluster_High-Low",
        "cluster_Low-High",
        "cluster_Low-Low",
        "storey_lag",
    ],
].explore(
    "abs_rel_error",
    tiles="CartoDB positron",
    style_kwds={"fillOpacity": 0.5},
    colormap="RdYlGr",
    legend=True,
    name="Prediction Errors",
)
